In [48]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import json
import sqlite3
data = json.load(open('/home/student/ttngo/SpotifyDataProject/data/mpd.slice.0-999.json'))
dfp = pd.DataFrame(data['playlists']) #get the dataframe from key of dictionary json

**Tables to create:** playlist able(pid, name, ....), tracks table( posiition, artist_name, track_uri,....), track_playlist  table(playlist_id, track_id,pos), artists table(artist_uri, artst_name), albums table(album_id, album_name)

In [49]:
conn = sqlite3.connect('data.db')
cur = conn.cursor()

## 1. PLaylists Table 

In [50]:
#CREATE
create_playlist_table = '''
CREATE TABLE IF NOT EXISTS Playlists (
    pid INT PRIMARY KEY,
    name TEXT,
    collaborative TEXT,
    modified_at INT,
    num_tracks INT,
    num_albums INT,
    num_followers INT
    )
    '''
cur.execute(create_playlist_table)
conn.commit()
print("Table 'Playlists' created successfully.")

Table 'Playlists' created successfully.


In [51]:
#INSERT VALUES
# Prepare data for tracks insertion
playlists_data=[]
for playlist in data['playlists']:
    playlists_data.append((
        playlist['name'],
        playlist['collaborative'],
        playlist['pid'],
        playlist['modified_at'],
        playlist['num_tracks'],
        playlist['num_albums'],
        playlist['num_followers']
    ))

#Inserting each keys in the tracks_data into the table
insert_query = '''
INSERT OR IGNORE INTO Playlists (name, collaborative, pid, modified_at, num_tracks, num_albums, num_followers)
VALUES (?, ?, ?, ?, ?, ?, ?)
'''

cur.executemany(insert_query, playlists_data)
conn.commit()
conn.close()

In [52]:
# RETREIVE THE DATA
conn = sqlite3.connect('data.db')
cur = conn.cursor()

# Execute a query to retrieve all rows from the playlist_names table
cur.execute('SELECT * FROM Playlists LIMIT 5')

# Fetch all results of the query
rows = cur.fetchall()

# Print the rows
for row in rows:
    print(row)
# Close the connection
cur.close()
conn.close()

(0, 'Throwbacks', 'false', 1493424000, 52, 47, 1)
(1, 'Awesome Playlist', 'false', 1506556800, 39, 23, 1)
(2, 'korean ', 'false', 1505692800, 64, 51, 1)
(3, 'mat', 'false', 1501027200, 126, 107, 1)
(4, '90s', 'false', 1401667200, 17, 16, 2)


In [53]:
#CHECK THE SIZE OF TABLE (NUMBER OF ROWS)
conn = sqlite3.connect('draft.db')
# Create a cursor object using the connection
cur = conn.cursor()

cur.execute('SELECT COUNT(*) FROM Playlists')
row_count = cur.fetchone()[0]

# Print the number of rows
print(f'Number of rows in the Tracks table: {row_count}')

# Close the cursor and connection
cur.close()
conn.close()

Number of rows in the Tracks table: 1000


## 2. TRACKS TABLES

In [54]:
#CREATE TABLE
conn = sqlite3.connect('data.db')
cur=conn.cursor()

#Create table
create_playlist_table = '''
CREATE TABLE IF NOT EXISTS TrackS (
    track_uri TEXT PRIMARY KEY,
    artist_name TEXT,
    artist_uri TEXT,
    track_name TEXT,
    album_name TEXT,
    album_uri TEXT,
    duration_ms INT
)
'''
cur.execute(create_playlist_table)
conn.commit()

In [56]:
# INSERT TABLE

tracks_data=[]
for playlist in data["playlists"]:
    for track in playlist["tracks"]:
        tracks_data.append((
            track["track_uri"], 
            track["artist_name"], 
            track["artist_uri"], 
            track["track_name"], 
            track["album_name"], 
            track["album_uri"], 
            track["duration_ms"]
        ))
        
insert_track = '''
INSERT OR IGNORE INTO Tracks (track_uri, artist_name, artist_uri, track_name, album_name, album_uri, duration_ms)
VALUES (?, ?, ?, ?, ?, ?, ?)
'''

cur.executemany(insert_track, tracks_data)
conn.commit()
conn.close()

In [57]:
#CHECK ROWS
conn = sqlite3.connect('data.db')

# Create a cursor object using the connection
cur = conn.cursor()

cur.execute('SELECT COUNT(*) FROM Tracks')
row_count = cur.fetchone()[0]

# Print the number of rows
print(f'Number of rows in the Tracks table: {row_count}')

# Close the cursor and connection
cur.close()
conn.close()

Number of rows in the Tracks table: 34443


In [58]:
# RETREIVE THE DATA
conn = sqlite3.connect('data.db')
cur = conn.cursor()

# Execute a query to retrieve all rows from the playlist_names table
cur.execute('SELECT * FROM Tracks LIMIT 2')

# Fetch all results of the query
rows = cur.fetchall()

# Print the rows
for row in rows:
    print(row)
# Close the connection
cur.close()
conn.close()

('spotify:track:0UaMYEvWZi0ZqiDOoHU3YI', 'Missy Elliott', 'spotify:artist:2wIVse2owClT7go1WT98tk', 'Lose Control (feat. Ciara & Fat Man Scoop)', 'The Cookbook', 'spotify:album:6vV5UrXcfyQD1wu4Qo2I9K', 226863)
('spotify:track:6I9VzXrHxO9rA9A5euc8Ak', 'Britney Spears', 'spotify:artist:26dSoYclwsYLMAKD3tpOr4', 'Toxic', 'In The Zone', 'spotify:album:0z7pVBGOD7HCIB7S8eLkLI', 198800)


## 3. Playlists_Tracks Table
**NOTE:** will have pid, tracks_uri, pos . (pid,track_uri) will be composite keys because a track can be appeared in multiple playlists. 

In [59]:
#CREATE
conn = sqlite3.connect('data.db')
cur = conn.cursor()

create_playtrack_table = '''
CREATE TABLE IF NOT EXISTS Playlists_Tracks (
    pid INT,
    track_uri TEXT,
    pos INT,
    FOREIGN KEY(pid) REFERENCES Playlists(pid),
    FOREIGN KEY(track_uri) REFERENCES Tracks(track_uri),
    PRIMARY KEY(pid, track_uri)
)
'''
cur.execute(create_playtrack_table)

In [60]:
#INSERT VALUES INTO PLAYLISTS_TRACKS
playlist_track_data=[]
for playlist in data["playlists"]:
    for track in playlist["tracks"]:
        playlist_track_data.append((
            playlist['pid'],
            track['track_uri'],
            track['pos']
        ))
        
insert_track = '''
INSERT OR IGNORE INTO Playlists_Tracks (pid, track_uri, pos)
VALUES (?, ?, ?)
'''

cur.executemany(insert_track, playlist_track_data)
conn.commit()
cur.close()
conn.close()

In [61]:
#CHECK ROWS
conn = sqlite3.connect('data.db')
cur = conn.cursor()

cur.execute('SELECT COUNT(*) FROM Playlists_Tracks')
row_count = cur.fetchone()[0]

# Print the number of rows
print(f'Number of rows in the Tracks table: {row_count}')

# Close the cursor and connection
conn.commit()
cur.close()
conn.close()

Number of rows in the Tracks table: 66721
